In [ ]:
import torch
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt

In [ ]:
batch_size = 32
kwargs = {}

In [ ]:
train_loader = torch.utils.data.DataLoader(datasets.MNIST('data', train=True, download=True, transform=transforms.Compose([
                           transforms.ToTensor()
                       ])), batch_size=batch_size, shuffle=True, **kwargs)

In [ ]:
test_loader = torch.utils.data.DataLoader(datasets.MNIST('data', train=False, 
                                                          download=True, 
                                                          transform=transforms.Compose([
                           transforms.ToTensor()
                       ])), 
                                                       batch_size=batch_size, shuffle=True, **kwargs)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(784, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 10)
    
    def forward(self, x):
        x = x.view(-1, 784)
        x = nn.LeakyReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [ ]:
model = Model()

In [ ]:
lr = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [ ]:
def train(model, train_loader, optimizer, epoch):
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

In [ ]:
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
 for epoch in range(1, 31):
        print('Epoch {}'.format(epoch))
        train(model, train_loader, optimizer, epoch)
        test(model, test_loader)